# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
import tqdm

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))
    print("Num file:", len(file_path_list))

/home/workspace
Num file: 30


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print("Num record data:", sum(1 for line in f))

Num record data: 6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("create keyspace if not exists event_database WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }")
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('event_database')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4




In [8]:
# Create table artist_song_title_length with session_id is partition key because the querie will filter by this column. item_in_session is clustering key to help make up a unique key.
query = "CREATE TABLE IF NOT EXISTS artist_song_title_length "
query = query + "(session_id int, item_in_session int, artist text, song_title text, song_length float, primary key (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)  

In [9]:
# Insert data to artist_song_title_length table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in tqdm.tqdm(csvreader):
        query = "insert into artist_song_title_length (session_id, item_in_session, artist, song_title, song_length) "
        query = query + "values (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

6820it [00:10, 654.61it/s]


In [10]:
# Select required data
query = "select artist, song_title, song_length from artist_song_title_length where item_in_session=4 and session_id=338"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [12]:
# Create table name_song_user with user_id, session_id are partition keys and item_in_session is sorted key
query = "CREATE TABLE IF NOT EXISTS name_song_user "
query = query + "(user_id int, session_id int, item_in_session int, artist text, song_title text, first_name text, last_name text, primary key ((user_id, session_id), item_in_session))"

try:
    session.execute(query)
except Exception as e:
    print(e)
    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in tqdm.tqdm(csvreader):
        query = "insert into name_song_user (user_id, session_id, item_in_session, artist, song_title, first_name, last_name) "
        query = query + "values (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
                    
            
query = "select artist, song_title, first_name, last_name from name_song_user where user_id=10 and session_id=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

6820it [00:08, 812.61it/s]

Row(artist='Down To The Bone', song_title="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')
Row(artist='Three Drives', song_title='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist='Sebastien Tellier', song_title='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [14]:
# Create table name_song with song_title is partition key and user_id is clustering key
query = "CREATE TABLE IF NOT EXISTS name_song  "
query = query + "(song_title text, user_id int, first_name text, last_name text, primary key (song_title, user_id))"

try:
    session.execute(query)
except Exception as e:
    print(e)
    
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in tqdm.tqdm(csvreader):
        query = "insert into name_song (song_title, user_id, first_name, last_name) "
        query = query + "values (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
                    
            
query = "select first_name, last_name from name_song where song_title='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)                    

6820it [00:08, 778.12it/s]

Row(first_name='Jacqueline', last_name='Lynch')
Row(first_name='Tegan', last_name='Levine')
Row(first_name='Sara', last_name='Johnson')


### Drop the tables before closing out the sessions

In [15]:
## TO-DO: Drop the table before closing out the sessions
query = "DROP TABLE IF EXISTS artist_song_title_length"

try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS name_song_user"

try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS name_song"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [16]:
query = 'drop keyspace event_database'
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [17]:
session.shutdown()
cluster.shutdown()